# CartoonGAN 논문의 모델 구조
<img src="https://aruie.github.io/assets/post/191114-01.png" width=700>


# generator
- 초기화 단계 : 10 epoch만큼 먼저 훈련

## input
- resizing된 사진 (resizing 함수를 이용하여 300x300으로 미리 resizing)

In [ ]:
# Google Drive mount
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# import
import os
import matplotlib.pyplot as plt
import PIL
import PIL.Image
import tensorflow as tf
from matplotlib.image import imread
import cv2
import numpy as np

In [ ]:
# 사진 있는 디렉토리 위치
photo_path = '/content/drive/MyDrive/GoogleColab/CartoonGAN_Project/data/photo_data'

# 사진 파일명 리스트 저장
photoName_list = os.listdir(photo_path)
print(photoName_list[:10])  # 앞의 10개만 파일명 보기
print(len(photoName_list))  # 사진 총 몇 장 있는 지 보기

## 모델링

In [ ]:
# 네트워크를 만들 땐
# 1. sequential 모델을 사용한 네트워크
# 2. 함수형 API를 사용한 네트워크

# 2가지 방법으로 만들 수 있다.

# 1번 방법 ->  일렬로 층을 쌓은 네트워크를 빠르게 만들 때 사용하기 좋다.
# 2번 방법 -> 한 층의 출력이 여러 개의 별도의 층으로 전달되는 네트워크를 만들 때 유연성있게 만들기 좋다.
#          -> 심층 신경망 구조를 설계하는데 자유롭다.

In [ ]:
# 참고 : https://www.tensorflow.org/api_docs
from keras import layers
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, Activation, Dropout, ReLU, Softmax, LeakyReLU, UpSampling2D
from keras.models import Model
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy, MAE, MeanSquaredError
from keras.metrics import Accuracy, CategoricalCrossentropy, MeanSquaredError
from keras.optimizers import Adam, Optimizer, RMSprop, SGD
from keras.regularizers import l1, l2

In [ ]:
# residual block은 똑같은 구조가 generator안에 8번 반복되므로 함수로 따로 만들자.
def generator_residual_block(x):
  shortcut = x
  x = Conv2D(kernel_size = 3,
            filters = 256,
            strides = 1,
            padding = "same"
            )(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(kernel_size = 3,
            filters = 256,
            strides = 1,
            padding = "same")(x)
  x = BatchNormalization()(x)
  x = layers.Add()([x, shortcut])  # identity shortcut connection  # elementwise sum
  
  return x

up-convolution 영역
1. UpSampling2D + Conv2D
2. Conv2DTranspose
- 두 가지 방법 다 사용해보고 어떤 것이 좋은 결과가 나오는 지 직접 확인해보고 결정해야함


- UpSampling2D docs 참고 : https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling2D
- Conv2DTranspose docs 참고 : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose



## 방법 1 - UpSampling2D + Conv2D 이용

In [ ]:
# 우선 input에 사진 이미지를 넣지 않고 숫자만 넣고
# 틀만 짜보자.

input_shape = (300, 300, 3)

input_layer = Input(shape = input_shape)  # resizing된 사진 넣기

# flat-convolution 영역
x = Conv2D(kernel_size = 7,
          filters = 64,
          strides = 1,
          padding = "same"
          )(input_layer)
x = BatchNormalization()(x)
x = ReLU()(x)

# down-convolution 영역
x = Conv2D(kernel_size = 3,
          filters = 128,
          strides = 2,
          padding = "same"
          )(x)

model = Model(inputs = input_layer,
              outputs = x,
              name = "generator")
x = Conv2D(kernel_size = 3,
          filters = 128,
          strides = 1,
          padding = "same"
          )(x)
x = BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(kernel_size = 3,
          filters = 256,
          strides = 2,
          padding = "same"
          )(x)
x = Conv2D(kernel_size = 3,
          filters = 256,
          strides = 1,
          padding = "same"
          )(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# 8 residual block 영역
x = generator_residual_block(x)
x = generator_residual_block(x)
x = generator_residual_block(x)
x = generator_residual_block(x)
x = generator_residual_block(x)
x = generator_residual_block(x)
x = generator_residual_block(x)
x = generator_residual_block(x)

# up-convolution 영역
x = Conv2D(kernel_size = 3,
           filters = 256,
           strides = 1/2,
           padding = "same"
          )(x)
x = Conv2D(kernel_size = 3,
          filters = 128,
          strides = 1,
          padding = "same"
          )(x)
x = BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(kernel_size = 3,
           filters = 64,
           strides = 1/2,
           padding = "same"
          )(x)
x = Conv2D(kernel_size = 3,
          filters = 64,
          strides = 1,
          padding = "same"
          )(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# output layer 영역
x = Conv2D(kernel_size = 7,
          filters = 3,
          strides = 1,
          padding = "same"
          )(x)

In [ ]:
model.summary()

## 방법2 - Conv2DTranspose 이용

# discriminator

## input
- 사진
- 엣지 smoothing된 만화 이미지
- 카툰화된 사진

In [ ]:
# 우선 input에 사진 이미지를 넣지 않고 숫자만 넣고
# 틀만 짜보자.

In [ ]:
input_layer = Input((300, 300, 3))
x = Conv2D(kernel_size = 3,
           filters = 32,
           strides = 1,
           padding = "same"
           )(input_layer)
x = LeakyReLU()(x)

x = Conv2D(kernel_size = 3,
           filters = 64,
           strides = 2,
           padding = "same"
           )(x)
x = LeakyReLU()(x)
x = Conv2D(kernel_size = 3,
           filters = 128,
           strides = 1,
           padding = "same"
           )(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(kernel_size = 3,
           filters = 128,
           strides = 2,
           padding = "same"
           )(x)
x = LeakyReLU()(x)
x = Conv2D(kernel_size = 3,
           filters = 256,
           strides = 1,
           padding = "same"
           )(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(kernel_size = 3,
           filters = 256,
           strides = 1,
           padding = "same"
           )(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(kernel_size = 3,
           filters = 1,
           strides = 1,
           padding = "same"
           )(x)